In [11]:
import kagglehub

path = kagglehub.dataset_download("dataceo/sales-and-customer-data")
print("Caminho do dataset:", path)

import os

os.listdir(path)

import pandas as pd

# Caminhos completos dos arquivos
caminho_sales = os.path.join(path, "sales_data.csv")
caminho_customers = os.path.join(path, "customer_data.csv")

# Leitura dos dados
df_sales = pd.read_csv(caminho_sales)
df_customers = pd.read_csv(caminho_customers)

# Visualizar as 5 primeiras linhas de cada um
print("Vendas:")
display(df_sales.head())

print("\nClientes:")
display(df_customers.head())

# Estrutura dos dados de vendas
print("Estrutura do df_sales:")
print(df_sales.info())
print(df_sales.columns)

# Estrutura dos dados de clientes
print("\nEstrutura do df_customers:")
print(df_customers.info())
print(df_customers.columns)

# Merge dos DataFrames com base no customer_id
df_full = pd.merge(df_sales, df_customers, on="customer_id", how="inner")

# Visualizar as primeiras linhas do DataFrame completo
print("DataFrame unido (vendas + clientes):")
df_full.head()

import sqlite3

# Criar conexão SQLite em memória (não salva em arquivo)
conn = sqlite3.connect(":memory:")

# Salvar o DataFrame como tabela SQL chamada 'clientes_vendas'
df_full.to_sql("clientes_vendas", conn, index=False, if_exists="replace")

# Verificar se a tabela foi criada com sucesso
consulta = pd.read_sql_query("SELECT * FROM clientes_vendas LIMIT 5", conn)
display(consulta)

# 1. Total gasto por cliente
query1 = """
SELECT customer_id,
       SUM(quantity * price) AS total_gasto
FROM clientes_vendas
GROUP BY customer_id
ORDER BY total_gasto DESC
LIMIT 10;
"""
print("Top 10 clientes por total gasto:")
display(pd.read_sql_query(query1, conn))

# 2. Ticket médio por categoria
query2 = """
SELECT category,
       AVG(quantity * price) AS ticket_medio
FROM clientes_vendas
GROUP BY category
ORDER BY ticket_medio DESC;
"""
print("Ticket médio por categoria:")
display(pd.read_sql_query(query2, conn))

# 3. Total de vendas por forma de pagamento
query3 = """
SELECT payment_method,
       COUNT(*) AS total_transacoes
FROM clientes_vendas
GROUP BY payment_method
ORDER BY total_transacoes DESC;
"""
print("Total de transações por forma de pagamento:")
display(pd.read_sql_query(query3, conn))

# 1. Gasto médio por gênero
query1 = """
SELECT gender,
       ROUND(AVG(quantity * price), 2) AS media_gasto
FROM clientes_vendas
GROUP BY gender
ORDER BY media_gasto DESC;
"""
print("Gasto médio por gênero:")
display(pd.read_sql_query(query1, conn))

# 2. Total de gasto por faixa etária
query2 = """
SELECT
    CASE
        WHEN age < 20 THEN 'Menor de 20'
        WHEN age BETWEEN 20 AND 29 THEN '20-29'
        WHEN age BETWEEN 30 AND 39 THEN '30-39'
        WHEN age BETWEEN 40 AND 49 THEN '40-49'
        ELSE '50+'
    END AS faixa_etaria,
    ROUND(SUM(quantity * price), 2) AS total_gasto
FROM clientes_vendas
WHERE age IS NOT NULL
GROUP BY faixa_etaria
ORDER BY total_gasto DESC;
"""
print("Total de gasto por faixa etária:")
display(pd.read_sql_query(query2, conn))



Caminho do dataset: C:\Users\lucas\.cache\kagglehub\datasets\dataceo\sales-and-customer-data\versions\1
Vendas:


,invoice_no,customer_id,category,quantity,price,invoice_date,shopping_mall
0,I138884,C241288,Clothing,5,1500.40,05-08-2022,Kanyon
1,I317333,C111565,Shoes,3,1800.51,12-12-2021,Forum Istanbul
2,I127801,C266599,Clothing,1,300.08,09-11-2021,Metrocity
3,I173702,C988172,Shoes,5,3000.85,16-05-2021,Metropol AVM
4,I337046,C189076,Books,4,60.60,24-10-2021,Kanyon



Clientes:


,customer_id,gender,age,payment_method
0,C241288,Female,28.0,Credit Card
1,C111565,Male,21.0,Debit Card
2,C266599,Male,20.0,Cash
3,C988172,Female,66.0,Credit Card
4,C189076,Female,53.0,Cash


Estrutura do df_sales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   invoice_no     99457 non-null  object 
 1   customer_id    99457 non-null  object 
 2   category       99457 non-null  object 
 3   quantity       99457 non-null  int64  
 4   price          99457 non-null  float64
 5   invoice_date   99457 non-null  object 
 6   shopping_mall  99457 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 5.3+ MB
None
Index(['invoice_no', 'customer_id', 'category', 'quantity', 'price',
       'invoice_date', 'shopping_mall'],
      dtype='object')

Estrutura do df_customers:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   customer_id     99457 non-null  object 
 1 

...

,payment_method,total_transacoes
0,Cash,44447
1,Credit Card,34931
2,Debit Card,20079


Gasto médio por gênero:


,gender,media_gasto
0,Male,2534.05
1,Female,2525.25


Total de gasto por faixa etária:


,faixa_etaria,total_gasto
0,50+,95769215.18
1,40-49,49172447.77
2,20-29,48987084.11
3,30-39,48260455.66
4,Menor de 20,9039330.06
